In [12]:
from shapely.geometry import Point, MultiLineString
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from geopy.geocoders import Nominatim
%matplotlib inline 

## get crashes_df from the API and clean

In [ ]:
endpoint_crashes = 'https://data.cityofchicago.org/resource/85ca-t3if.geojson?$select=crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,first_crash_type,trafficway_type,lane_cnt,road_defect,prim_contributory_cause,sec_contributory_cause,most_severe_injury,injuries_total,injuries_fatal,injuries_incapacitating,injuries_non_incapacitating,injuries_reported_not_evident,injuries_no_indication,injuries_unknown,crash_hour,crash_day_of_week,crash_month,street_no,street_direction,street_name,latitude,longitude,location&$limit=650000'
res = requests.get(endpoint_crashes)
res = res.json()

In [ ]:
crashes_df = gpd.GeoDataFrame.from_features(res['features'])

In [ ]:
crashes_df.info()

In [ ]:
type(crashes_df)

In [ ]:
#filter out crashes where there was debris on roadway
crashes_df = crashes_df.loc[crashes_df['road_defect'] != 'DEBRIS ON ROADWAY']

In [ ]:
#filter out crashes where the traffic control device was functioning improperly or not functioning
device_not_working = ['FUNCTIONING IMPROPERLY','NOT FUNCTIONING']
crashes_df = crashes_df.loc[~crashes_df['device_condition'].isin(device_not_working)]
#crashes.loc[crashes['crash_record_id'].isin(crash_id_peds)]

In [ ]:
#there is one row in there with POINT of (0,0) and it throws off plotting the map later on, so get rid of it
crashes_df = crashes_df[crashes_df['geometry'].x != 0]

In [ ]:
#concat full address
crashes_df['full_address'] = crashes_df['street_no'] + ' ' + crashes_df['street_direction'] + ' ' + crashes_df['street_name'] + ' CHICAGO IL'

In [ ]:
#filter out 5 rows with null full_address
crashes_df = crashes_df[crashes_df['full_address'].notna()]

In [ ]:
#how many null geometries are there? 3877
len(crashes_df[crashes_df['geometry'].isna()])

In [ ]:
#geopy code. this takes an address and generates long/lat
#checking to see if it works for one single row before applying to larger set
geolocator = Nominatim(user_agent="colin")
location = geolocator.geocode(crashes_df.loc[1,'full_address'])
print(location.address)
print((location.longitude, location.latitude))

In [ ]:
#check what data type is generated
type(location.longitude)

In [ ]:
#could try counter, progress bar
#this code looks at every row of crashes_df. it tries to generate long/lat using the full_address. if it can't, it prints the index. if it can, it makes it into a POINT geometry and puts it in the geometry column in the row it got the address from
for index, row in crashes_df.iterrows():
    if row.geometry is None:
        try:
            geolocator = Nominatim(user_agent="colin")
            location = geolocator.geocode(row['full_address'])
            crashes_df.at[index, 'geometry'] = Point((location.longitude, location.latitude))
        except: 
            print(index)

In [ ]:
#how many null geometries are there now that we populated missing ones from the address?
#there are still 2097. that must be how many weird addresses there are that it can't generate a lat/long for.
#addresses like '1 w parking lot a' 
crashes_df[crashes_df['geometry'].isna()]

In [ ]:
#remove null geometries that we could not generate a POINT for from the address
crashes_df = crashes_df[crashes_df['geometry'].notna()]

In [ ]:
crashes_df.info()

In [ ]:
#convert injury counts from strings to integers
crashes_df['injuries_fatal'] = pd.to_numeric(crashes_df['injuries_fatal'])
crashes_df['injuries_non_incapacitating'] = pd.to_numeric(crashes_df['injuries_non_incapacitating'])
crashes_df['injuries_no_indication'] = pd.to_numeric(crashes_df['injuries_no_indication'])
crashes_df['injuries_incapacitating'] = pd.to_numeric(crashes_df['injuries_incapacitating'])
crashes_df['injuries_unknown'] = pd.to_numeric(crashes_df['injuries_unknown'])
crashes_df['injuries_total'] = pd.to_numeric(crashes_df['injuries_total'])
crashes_df['injuries_reported_not_evident'] = pd.to_numeric(crashes_df['injuries_reported_not_evident'])

In [ ]:
#combine "injuries reported not evident" and "injuries no indication" into category called "injuries not significant"
crashes_df['injuries_not_significant'] = crashes_df['injuries_reported_not_evident'] + crashes_df['injuries_no_indication']

In [ ]:
#check that 2nd and 3rd add up to 1st
crashes_df[['injuries_not_significant', 'injuries_reported_not_evident', 'injuries_no_indication']]

In [ ]:
crashes_df = crashes_df.reset_index(drop=True)

In [ ]:
#save crashes_df as geojson
crashes_df.to_file("data/crashes_cleaned.geojson", driver='GeoJSON')

## get pedestrians from the people API, clean, export to CSV

In [ ]:
endpoint_people = 'https://data.cityofchicago.org/resource/u6pd-qa9d.json?person_type=PEDESTRIAN&$limit=16000'
res = requests.get(endpoint_people)
res = res.json()
peds_df = pd.DataFrame(res)

In [ ]:
peds_df['injury_classification'].value_counts()

In [ ]:
#remove peds who were impaired by drugs or alcohol or both
#DON'T filter in ['NORMAL','UNKNOWN','REMOVED BY EMS','OTHER','EMOTIONAL'] b/c you lose 1000 NaNs 
#instead, filter OUT with impaired_list and ~

impaired_list = ['IMPAIRED - ALCOHOL',
'HAD BEEN DRINKING',
'IMPAIRED - DRUGS',
'IMPAIRED - ALCOHOL AND DRUGS',
'FATIGUED/ASLEEP',
'ILLNESS/FAINTED',
'MEDICATED']
peds_df = peds_df.loc[~peds_df['physical_condition'].isin(impaired_list)]

In [ ]:
#remove pedpedal_action "INTOXICATED"
peds_df = peds_df.loc[peds_df['pedpedal_action'] != 'INTOXICATED PED/PEDAL']

In [ ]:
peds_df = peds_df.reset_index(drop=True)

In [ ]:
#create new value called "NO SIGNIFICANT INJURY" in the injury_classification column by combining "REPORTED, NOT EVIDENT" and "NO INDICATION OF INJURY"
#this makes more semantic sense for our purposes (the 2 original categories probably exist for police/legal purposes)
for index, row in peds_df.iterrows():
    if row.injury_classification == "REPORTED, NOT EVIDENT":
        peds_df.loc[index, 'injury_classification'] = 'NO SIGNIFICANT INJURY'
    elif row.injury_classification == "NO INDICATION OF INJURY":
        peds_df.loc[index, 'injury_classification'] = 'NO SIGNIFICANT INJURY'

In [ ]:
#make sure it worked
peds_df['injury_classification'].value_counts()

In [ ]:
peds_df.to_csv('data/peds_cleaned_df.csv')

## get cyclists from the people API, clean, export to CSV

In [ ]:
endpoint_people = 'https://data.cityofchicago.org/resource/u6pd-qa9d.json?person_type=BICYCLE&$limit=16000'
res = requests.get(endpoint_people)
res = res.json()
cyclists_df = pd.DataFrame(res)

In [ ]:
cyclists_df.info()

In [ ]:
#remove cyclists who were impaired by drugs or alcohol or both
#DON'T filter in ['NORMAL','UNKNOWN','REMOVED BY EMS','OTHER','EMOTIONAL'] b/c you lose NaNs 
#instead, filter OUT with impaired_list and ~

impaired_list = ['IMPAIRED - ALCOHOL',
'HAD BEEN DRINKING',
'IMPAIRED - DRUGS',
'IMPAIRED - ALCOHOL AND DRUGS',
'FATIGUED/ASLEEP',
'ILLNESS/FAINTED',
'MEDICATED']
cyclists_df = cyclists_df.loc[~cyclists_df['physical_condition'].isin(impaired_list)]

In [ ]:
#remove pedpedal_action "INTOXICATED"
cyclists_df = cyclists_df.loc[cyclists_df['pedpedal_action'] != 'INTOXICATED PED/PEDAL']

In [ ]:
cyclists_df = cyclists_df.reset_index(drop=True)

In [ ]:
for index, row in cyclists_df.iterrows():
    if row.injury_classification == "REPORTED, NOT EVIDENT":
        cyclists_df.loc[index, 'injury_classification'] = 'NO SIGNIFICANT INJURY'
    elif row.injury_classification == "NO INDICATION OF INJURY":
        cyclists_df.loc[index, 'injury_classification'] = 'NO SIGNIFICANT INJURY'

In [ ]:
cyclists_df['injury_classification'].value_counts()

In [ ]:
cyclists_df.to_csv('data/cyclists_cleaned_df.csv')

# crash_date cleaning

In [ ]:
#cleaning crash_date
test_time = crashes_df.loc[15,'crash_date']
type(test_time)
#it's a string so convert to datetime

In [ ]:
#try it on one single value
timestamp=pd.to_datetime(test_time, format = '%Y/%m/%d %H:%M:%S')

In [ ]:
#gives us a Timestamp
type(timestamp)
timestamp

In [ ]:
#can use Timestamp methods on it
#Timestamp is pandas equivalent of Python datetime 
#https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.html
timestamp.month_name()

### making this a Timestamp screwed up my folium map. folium map wants a string
### having as Timestamp throws TypeError: Object of type Timestamp is not JSON serializable
#convert crash_date of whole df
#crashes_df['crash_date'] = pd.to_datetime(crashes_df['crash_date'], format = '%Y/%m/%d %H:%M:%S')

In [ ]:
#crash day of week, crash hour and crash month look fine as far as their range
crashes_df['crash_day_of_week'].value_counts()

In [ ]:
#injury columns to integers
#leave posted_speed_limit, lane_cnt as objects, they are more like categories than numerical measures
#leave crash_month, crash_day_of_week and crash_hour as objects for now, not sure what to make them

In [ ]:
crashes_df.dtypes

# try to get lane count from street center lines

In [16]:
crashes_df = gpd.read_file('data/crashes_cleaned.geojson')
crashes_df.set_crs(crs='EPSG:4326', inplace=True)

,injuries_fatal,injuries_non_incapacitating,crash_record_id,injuries_incapacitating,injuries_no_indication,latitude,street_no,injuries_unknown,device_condition,crash_date,...,prim_contributory_cause,sec_contributory_cause,street_direction,posted_speed_limit,injuries_total,lane_cnt,full_address,no_significant_injury,injuries_not_significant,geometry
0,NaN,NaN,0edc78f89df5b72ddaa1c1f567e229ad39c0e098953a9d...,NaN,NaN,41.873520168,3357,NaN,NO CONTROLS,2022-08-09T01:40:00,...,FAILING TO REDUCE SPEED TO AVOID CRASH,NOT APPLICABLE,W,30,NaN,None,3357 W HARRISON ST CHICAGO IL,NaN,NaN,POINT (-87.71064 41.87352)
1,0.0,0.0,49ff3f04d16f5a71e6d66436c94f6acfd939c20d6c3651...,0.0,2.0,41.831835811,3410,0.0,FUNCTIONING PROPERLY,2022-08-09T00:45:00,...,FOLLOWING TOO CLOSELY,NOT APPLICABLE,S,30,0.0,None,3410 S DAMEN AVE CHICAGO IL,2.0,2.0,POINT (-87.67546 41.83184)
2,NaN,NaN,e580e89f187525bf685101a36fc64df499a72be926d5a9...,NaN,NaN,41.838371536,130,NaN,FUNCTIONING PROPERLY,2022-08-08T23:55:00,...,FAILING TO YIELD RIGHT-OF-WAY,NOT APPLICABLE,W,35,NaN,None,130 W 31ST ST CHICAGO IL,NaN,NaN,POINT (-87.63002 41.83837)
3,0.0,0.0,cdd7c5d90668e4d1bda12805ad19cec305667643a88806...,0.0,1.0,41.861481407,2747,0.0,NO CONTROLS,2022-08-08T23:00:00,...,UNABLE TO DETERMINE,UNABLE TO DETERMINE,W,30,0.0,None,2747 W OGDEN AVE CHICAGO IL,1.0,1.0,POINT (-87.69494 41.86148)
4,0.0,0.0,e7c26ad2dd7f250b14acafe2d86265616f59f7c642a505...,0.0,2.0,41.837964154,3100,0.0,FUNCTIONING PROPERLY,2022-08-08T23:00:00,...,DISREGARDING TRAFFIC SIGNALS,DISREGARDING TRAFFIC SIGNALS,S,35,0.0,None,3100 S HALSTED ST CHICAGO IL,2.0,2.0,POINT (-87.64642 41.83796)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631400,0.0,0.0,964aaaeb569e364886cfbdf89ca73e09ca15cd916d87b2...,0.0,2.0,41.835886103,3132,0.0,NO CONTROLS,2014-06-25T19:00:00,...,IMPROPER OVERTAKING/PASSING,NOT APPLICABLE,S,30,0.0,None,3132 S PULASKI RD CHICAGO IL,2.0,2.0,POINT (-87.72447 41.83589)
631401,0.0,0.0,1d0232afecbdfd01968555aa956a688fd6f55a2bd1984f...,0.0,2.0,41.884016475,199,0.0,FUNCTIONING PROPERLY,2014-02-24T19:45:00,...,UNABLE TO DETERMINE,UNABLE TO DETERMINE,N,30,0.0,None,199 N SACRAMENTO BLVD CHICAGO IL,2.0,2.0,POINT (-87.70114 41.88402)
631402,0.0,1.0,957783a4787318f005a7dbc920e4c84cb9ac8aa7329a62...,0.0,1.0,41.760710194,7400,0.0,NO CONTROLS,2014-01-21T07:40:00,...,FAILING TO YIELD RIGHT-OF-WAY,NOT APPLICABLE,S,30,1.0,None,7400 S EXCHANGE AVE CHICAGO IL,1.0,1.0,POINT (-87.56195 41.76071)
631403,0.0,0.0,f62e27317feb174811cf4fefeb9fa1064fea6c0619a873...,0.0,2.0,41.885609917,415,0.0,NO CONTROLS,2014-01-18T18:14:00,...,UNABLE TO DETERMINE,UNABLE TO DETERMINE,W,30,0.0,None,415 W LAKE ST CHICAGO IL,2.0,2.0,POINT (-87.63876 41.88561)


In [17]:
crashes_df.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
crashes_df['geometry']

## load street center lines from geojson

In [59]:
street_center_lines_geo = gpd.read_file('data/street_center_lines.geojson')
# gpd.explode this would explode multi part geometries into multiple single part geometries

NotImplementedError: Multi-part geometries do not themselves provide the array interface

In [ ]:
street_center_lines.isna().sum()

In [6]:
from shapely import wkt
street_center_lines['the_geom'] = street_center_lines['the_geom'].apply(wkt.loads)
#gpd.GeoSeries.from_wkt(street_center_lines['the_geom'])


In [11]:
street_center_lines.explode('the_geom')['the_geom'].apply

<bound method Series.apply of 0        LINESTRING (-87.63187018233509 41.780809747581...
1        LINESTRING (-87.60548932047928 41.762229643527...
2        LINESTRING (-87.6838184819274 41.6972438444297...
3        LINESTRING (-87.64584189298007 41.693398400444...
4        LINESTRING (-87.67053671319235 41.679759274890...
                               ...                        
56333    LINESTRING (-87.55784956501411 41.657918209955...
56334    LINESTRING (-87.73902939370839 41.948344047485...
56335    LINESTRING (-87.73879459600205 41.947181951899...
56336    LINESTRING (-87.74339277679763 41.891415000932...
56337    LINESTRING (-87.68107407701358 41.863969346397...
Name: the_geom, Length: 56338, dtype: object>

In [13]:
gpd.GeoDataFrame(data=street_center_lines, geometry='the_geom', crs='EPSG:4326')

NotImplementedError: Multi-part geometries do not themselves provide the array interface

In [ ]:
#crashes_df.sjoin(street_center_lines, op = 'within')
gpd.sjoin(crashes_df, street_center_lines, op = 'within')

In [ ]:
import shapely
shapely.__version__

In [ ]:
street_center_lines = street_center_lines[['the_geom','CLASS']]
street_center_lines.loc[1,'the_geom']

## load street center lines from API 

In [ ]:
endpoint = 'https://data.cityofchicago.org/resource/pr57-gg9e.json'
res = requests.get(endpoint)
res = res.json()
street_center_lines_API = pd.DataFrame(res)

In [ ]:
street_center_lines_API

## load street center lines from CSV

In [3]:
street_center_lines = pd.read_csv('data/street_center_lines.csv')

In [ ]:
street_center_lines.info()

In [ ]:
street_center_lines.head()

In [ ]:
street_center_lines['the_geom'].str.extract('(\d{})\s')

In [20]:
classes = ['1','2','3','4']
street_center_lines = street_center_lines.loc[street_center_lines['CLASS'].isin(classes)].reset_index()

In [ ]:
#clean up
street_center_lines['CLASS'].value_counts()

In [27]:
#L_F_ADD, L_T_ADD etc etc are the RANGE OF ADDRESSES FOR THAT STREET. When I clicked on Carmen between Broadway
#and Glenwood, LFADD and RFADD were 1301 and 1300. LTADD and RTADD were 1359 and 1358.
city_addresses = street_center_lines[['L_F_ADD','L_T_ADD','R_F_ADD','R_T_ADD','PRE_DIR','STREET_NAM','STREET_TYP','CLASS']]

In [31]:
city_addresses['searchable_address'] = city_addresses['PRE_DIR'] + ' ' + city_addresses['STREET_NAM'] + ' ' + city_addresses['STREET_TYP']

/tmp/ipykernel_56141/3888918883.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_addresses['searchable_address'] = city_addresses['PRE_DIR'] + ' ' + city_addresses['STREET_NAM'] + ' ' + city_addresses['STREET_TYP']


In [33]:
city_addresses = city_addresses[['L_F_ADD','L_T_ADD','R_F_ADD','R_T_ADD','searchable_address','CLASS']]

In [51]:
city_addresses['min'] = city_addresses.min(axis=1)
city_addresses['max'] = city_addresses.max(axis=1)

/tmp/ipykernel_56141/3537348477.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  city_addresses['min'] = city_addresses.min(axis=1)
/tmp/ipykernel_56141/3537348477.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  city_addresses['max'] = city_addresses.max(axis=1)


In [ ]:
for index, row in city_addresses.iterrows():
    if row.L_F_ADD < L_T_ADD & R_F_ADD & R_T_ADD:
        city_addresses.loc[index, "min"] = L_F_ADD

for index, row in movies_df.iterrows():
    if row['duration'] <= 30:
        movies_df.loc[index, 'category'] = 'short'
    elif row['duration'] <= 120:
        movies_df.loc[index, 'category'] = 'average'
    else:
        movies_df.loc[index, 'category'] = 'long'

In [41]:
mins = city_addresses.idxmin(axis="columns")

for i, r in df.iterrows():
    mins[i] = list(r[r == r[mins[i]]].index)

TypeError: reduction operation 'argmin' not allowed for this dtype

In [52]:
city_addresses.head()

,L_F_ADD,L_T_ADD,R_F_ADD,R_T_ADD,searchable_address,CLASS,min,max
0,0,0,6228,6248,S YALE AVE,4,0,6248
1,7301,7359,7300,7358,S COTTAGE GROVE AVE,2,7300,7359
2,10801,10859,10800,10858,S CAMPBELL AVE,4,10800,10859
3,0,0,11026,11032,S SANGAMON ST,4,0,11032
4,1933,1959,1932,1958,W 118TH ST,4,1932,1959


In [22]:
crashes_df['searchable_address'] = crashes_df['street_direction'] + ' ' + crashes_df['street_name']

In [38]:
crashes_df['street_no'] = pd.to_numeric(crashes_df['street_no'])

In [46]:
null_lanes = crashes_df[crashes_df['lane_cnt'].isna()]

In [47]:
null_lanes.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 434861 entries, 0 to 631403
Data columns (total 32 columns):
 #   Column                         Non-Null Count   Dtype   
---  ------                         --------------   -----   
 0   injuries_fatal                 433881 non-null  float64 
 1   injuries_non_incapacitating    433881 non-null  float64 
 2   crash_record_id                434861 non-null  object  
 3   injuries_incapacitating        433881 non-null  float64 
 4   injuries_no_indication         433881 non-null  float64 
 5   latitude                       433643 non-null  object  
 6   street_no                      434861 non-null  int64   
 7   injuries_unknown               433881 non-null  float64 
 8   device_condition               434861 non-null  object  
 9   crash_date                     434861 non-null  object  
 10  trafficway_type                434861 non-null  object  
 11  traffic_control_device         434861 non-null  object  
 12  road_def

In [53]:
null_lanes[['street_no','searchable_address','lane_cnt']]

,street_no,searchable_address,lane_cnt
0,3357,W HARRISON ST,None
1,3410,S DAMEN AVE,None
2,130,W 31ST ST,None
3,2747,W OGDEN AVE,None
4,3100,S HALSTED ST,None
...,...,...,...
631398,1298,W DEVON AVE,None
631400,3132,S PULASKI RD,None
631401,199,N SACRAMENTO BLVD,None
631402,7400,S EXCHANGE AVE,None


In [ ]:
from tqdm.notebook import tqdm
#wrap null_lanes.iterrows in tqdm
#creates progress bar

In [58]:
for null_index, null_row in null_lanes.iterrows():
    for index, row in city_addresses.iterrows():
        if row['min'] <= null_row.street_no <= row['max'] and null_row.searchable_address == row.searchable_address:
            null_lanes.at[null_index, 'lane_cnt'] = row.CLASS
            break

KeyboardInterrupt: 

In [ ]:
null_lanes[['street_no','searchable_address','lane_cnt']]

In [ ]:
 if row.street_no between 
        for row min and row.max and searchable_address = searchable_address :
        return class at 
                crashes_df.at[index, 'geometry'] = Point((location.longitude, location.latitude))